In [3]:
! pip install scipy numpy plotly pandas

ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle


In [2]:
import numpy as np
from scipy.stats import norm
from tqdm import tqdm # for progress bar
import plotly.graph_objects as go

import pickle
import pandas as pd
from datetime import datetime

# Main Code

## Algorithm Implementation
The theoretical channel capacity is C = 1/2 * log(1 + p/sigma^2) bits per channel used .

In [ ]:
# Theoretical Capacity (C) = 1/2 * log2(1 + A^2)
def theoreticalCapacity(P, _sigma=1):
    return 0.5 * np.log2(1 + P / _sigma**2)

def getMeanPower(p_x, x):
    # Calculate mean power as E[X^2] = sum(p(x) * x^2)
    return np.sum(p_x * x**2)

def getChannel(x, y, sigma=1):
    """
    Compute the transition probabilities P(y|x) for a Gaussian noise channel.

    Parameters:
        x (array): Input alphabet.
        y (array): Output alphabet.
        sigma (float): Noise standard deviation.

    Returns:
        p_y_given_x (2D array): Transition probability matrix P(y|x).
    """
    # Compute pairwise differences
    diff_x_y = (x.reshape(1, -1) - y.reshape(-1, 1))

    # Compute half-width of interval (dy)
    dy = (y[1] - y[0]) * 0.5 if len(y) > 1 else 0.5  # Handle single-element case

    # Compute probabilities using the standard normal CDF
    p_y_given_x = norm.cdf(diff_x_y + dy, scale=sigma) - norm.cdf(diff_x_y - dy, scale=sigma)

    # Normalize columns to sum to 1
    p_y_given_x /= p_y_given_x.sum(axis=0, keepdims=True)
    return p_y_given_x


def runAlgorithm(A=1, sigma=1, max_iter=1000, NX=500, NY=1000, tolerance=1e-6, epsilon=1e-12,
                 printInit=False, earlyStop=True, recordFrequency=50):
    """
    Blahut-Arimoto algorithm to compute channel capacity for a Gaussian channel.

    Parameters:
        A (float): Peak power constraint.
        sigma (float): Noise standard deviation.
        max_iter (int): Maximum number of iterations.
        NX (int): Number of discretized points for input alphabet.
        NY (int): Number of discretized points for output alphabet.
        tolerance (float): Convergence criterion.
        epsilon (float): Small value for numerical stability.
        printInit (bool): Whether to print initial parameters.
        earlyStop (bool): Stop early if convergence criterion is met.
        recordFrequency (int): Frequency of recording intermediate distributions.

    Returns:
        dict: Results including capacity, distributions, and other metrics.
    """
    # Discretize the input and output alphabets
    x = np.linspace(-A, A, NX)  # Input alphabet
    y = np.linspace(-A - 4 * sigma, A + 4 * sigma, NY)  # Output alphabet

    # Transition probabilities P(y|x)
    p_y_given_x = getChannel(x, y, sigma)

    # Step 1 - Uniform initial input distribution
    p_x = np.ones((NX, 1)) / NX

    # For recording intermediate results
    p_x_records = []
    p_y_records = []

    if printInit:
        print(f"A = {A}, sigma = {sigma}, max_iter = {max_iter}, NX = {NX}, NY = {NY}")
        print(f"Initial input distribution p(x): (size {p_x.shape}) max= {p_x.max()} min = {p_x.min()}")
        print(f"Input alphabet: (size = {x.shape}) max = {x.max()} min = {x.min()}")
        print(f"Output alphabet: (size = {y.shape}) max = {y.max()} min = {y.min()}")
        print(f"Transition probabilities p(y|x): (size {p_y_given_x.shape}) max = {p_y_given_x.max()} min = {p_y_given_x.min()}")

    # Blahut-Arimoto iterations
    for iter in tqdm(range(max_iter), desc="Iterations", ncols=50):
        # Compute P(y)
        p_y = np.dot(p_y_given_x, p_x)  # Add epsilon for numerical stability
        p_y /= np.sum(p_y)  # Normalize

        # Update q(x|y)
        q_x_given_y = (p_y_given_x * p_x.reshape(1, -1)).T / p_y.T  # Shape (NX, NY)
        q_x_given_y /= q_x_given_y.sum(axis=0, keepdims=True)  # Normalize rows

        # Update p(x)
        p_x_new = np.exp(np.sum(p_y_given_x * np.log(np.maximum(q_x_given_y.T, epsilon)), axis=0))
        p_x_new /= np.sum(p_x_new)  # Normalize

        # Check for convergence
        if earlyStop and np.max(np.abs(p_x_new - p_x)) < tolerance:
            print(f"Converged in {iter + 1} iterations.")
            break

        # Record intermediate results
        if iter % recordFrequency == 0:
            p_x_records.append(p_x_new.copy())
            p_y_records.append(p_y.copy())

        p_x = p_x_new

    # Compute channel capacity
    log_ratio = np.log(np.maximum(p_y_given_x / p_y[:, None], epsilon))
    capacity = np.sum(p_x.flatten() * np.sum(p_y_given_x * log_ratio, axis=0))

    # Compute mean power
    mean_power = getMeanPower(p_x, x)

    return {
        "A": A,
        "capacity": capacity,
        "theoretical_c": theoreticalCapacity(A, sigma),
        "x": x,
        "y": y,
        "p_x": p_x,
        "p_y": p_y,
        "q_x_given_y": q_x_given_y,
        "p_y_given_x": p_y_given_x,
        "p_x_records": p_x_records,
        "p_y_records": p_y_records,
        "iter": iter + 1,
        "mean_power": mean_power
    }


## Plotly Code (interactive and dynamic visualization)

In [ ]:
import plotly.graph_objects as go

def plot_A_vs_capacity(As, capacities):
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=As,
        y= capacities,
        mode='lines',
        name='Channel Capacity'
    ))

    fig.update_layout(
        title="Channel Capacity vs. Peak Power Constraint",
        xaxis_title="A (Peak Power Constraint)",
        yaxis_title="Channel Capacity (bits)",
        template="plotly_dark",
        showlegend=False,
        autosize=True,
    )

    fig.show()

    # Print results
    for i in range(len(As)):
        print(f"A = {As[i]:.2f} | Channel Capacity = {capacities[i]:.3f}")


def plot_capcity_vs_theoretical_capacity(As, capacities, theoretical_capacitys):
    fig = go.Figure()

    # Add the computed capacity
    fig.add_trace(go.Scatter(
        x=As,
        y=capacities,
        mode='lines',
        name='Computed Capacity'
    ))

    # Add the theoretical capacity
    fig.add_trace(go.Scatter(
        x=As,
        y=theoretical_capacitys,
        mode='lines',
        name='Theoretical Capacity'
    ))

    fig.update_layout(
        title="Channel Capacity vs. Peak Power Constraint",
        xaxis_title="A (Peak Power Constraint)",
        yaxis_title="Capacity (bits)",
        template="plotly_dark",
        showlegend=True,
        autosize=True,
    )

    fig.show()

    # Print results
    for i in range(len(As)):
        print(f"A = {As[i]:.2f} | Channel Capacity = {capacities[i]:.3f} Theoretical = {theoretical_capacitys[i]:.3f}")

def plot_A_vs_p_x(outputs):
    fig = go.Figure()

    for i, output in enumerate(outputs):
        fig.add_trace(go.Scatter(
            x=output["x"],    # x-axis: the input symbols (discretized x)
            y=output["p_x"],  # y-axis: the probability distribution p(x)
            mode='lines',
            name=f"A = {(output['A']):.2f}",
        ))

    fig.update_layout(
        title="Input Probability Distributions for Different A",
        xaxis_title="x (Input Symbol)",
        yaxis_title="p(x) (Probability Distribution)",
        template="plotly_dark",
        showlegend=True,
        autosize=True,
    )

    fig.show()


def plot_A_vs_p_y(outputs):
    fig = go.Figure()

    for i, output in enumerate(outputs):
        fig.add_trace(go.Scatter(
            x=output["y"],  # x-axis: the output symbols (discretized y)
            y=output["p_y"],  # y-axis: the probability distribution p(y)
            mode='lines',
            name=f"A = {As[i]:.2f}",
        ))

    fig.update_layout(
        title="Output Probability Distributions for Different A",
        xaxis_title="y (Output Symbol)",
        yaxis_title="p(y) (Probability Distribution)",
        template="plotly_dark",
        showlegend=True,
        autosize=True,
    )

    fig.show()


def plot_p_x_records(outputs, idx=-1):
    """   !!!!STATIC!!!
    Example usage: Assuming p_x_records is a list of probability distributions for each iteration
    p_x_records = [p_x_iteration_1, p_x_iteration_2, ..., p_x_iteration_n]"""

    fig = go.Figure()

    output = outputs[idx]

    for i, p_x in enumerate(output["p_x_records"]):
        fig.add_trace(go.Scatter(
            x=output["x"],    # x-axis: the input symbols (discretized x)
            y=p_x,  # y-axis: the probability distribution p(x)
            mode='lines',
            name=f"Record {(i+1)}",
        ))

    fig.update_layout(
        title="Input Probability Distributions Over Iterations",
        xaxis_title="x (Input Symbol)",
        yaxis_title="p(x) (Probability Distribution)",
        template="plotly_dark",
        showlegend=True,
        autosize=True,
    )

    fig.show()

def plot_p_x_records_dynamic(outputs, idx=-1):
    """Example usage: Assuming p_x_records is a list of probability distributions for each iteration
    p_x_records = [p_x_iteration_1, p_x_iteration_2, ..., p_x_iteration_n]"""

    # Create an empty figure
    fig = go.Figure()

    output = outputs[idx]

    # Add traces for each iteration (initially all traces hidden)
    for i, p_x in enumerate(output["p_x_records"]):
        fig.add_trace(go.Scatter(
            x=output["x"],  # x-axis: the input symbols (discretized x)
            y=p_x,  # y-axis: the probability distribution p(x)
            mode='lines',  # Connect the points with lines
            name=f"A = {output['A']:.2f}, Record {(i+1)}",  # Label for the iteration
            visible='legendonly',  # Initially set to not visible
        ))

    # Create the slider steps (one per iteration)
    steps = []
    for i in range(len(output["p_x_records"])):
        step = dict(
            method="update",
            args=[{"visible": [False] * len(output["p_x_records"])},  # Hide all traces
                  {"title": f"Record {(i+1)}"}],  # Update the title
        )
        step["args"][0]["visible"][i] = True  # Make the current iteration visible
        steps.append(step)

    # Add the slider to the layout
    fig.update_layout(
        title="Input Probability Distributions Over Iterations",
        xaxis_title="x (Input Symbol)",
        yaxis_title=f"p(x) (Probability Distribution)",
        template="plotly_dark",  # Optional: use a dark theme
        showlegend=True,
        sliders=[dict(
            currentvalue={"prefix": f"Iteration: ", "visible": True, "xanchor": "center"},
            steps=steps
        )]
    )

    # Show the dynamic plot
    fig.show()

    # print
    print("==========")
    print(f"Total Iterations: {(output['iter'])}")
    print(f"A = {output['A']:.2f}")
    print(f"Channel Capacity = {output['capacity']:.3f}")
    print(f"Theoretical Capacity = {output['theoretical_c']:.3f}")
    print(f"Mean power = {output['mean_power']:.3f}")


# Experiment

## Algorithm Run

In [ ]:
# Experiment set up

As = [10]
# As = [0.1, 1, 10]
# As = [0.1, 0.5, 1, 5, 10]
# As = np.arange(0.1, 10.0, 0.25)  # Step size of 0.25
# As = np.logspace(-1, 1.0, num=20, base=10)  # evenly spaced on log scale

# Lists to store output variables
capacities = []
meanPower_TheoreticalCapacities = [theoreticalCapacity(A) for A in As]
outputs = []

# Perform the experiment
print("+++++++++++")
print(f"Running Experiment: As = {As} \nNx = 1000, NY = 2000 \nTolerance = 10^-5")
print("+++++++++++")
for A in As:
    print("**************")
    print(f"Running experiment for A = {A:.2f} / {(np.max(As)):.2f}")
    print("-------------")
    output = runAlgorithm(A,NX=1000,NY=2000 ,max_iter=20000, earlyStop=True, recordFrequency=100, tolerance=1e-6)
    # store the simulation summary
    outputs.append(output)
    capacities.append(output['capacity'])


print("+++++++++++")
print("Simulation runs Complete")
print("++++++++++++")
# sanityCheck(outputs)

+++++++++++
Running Experiment: As = [10] 
Nx = 1000, NY = 2000 
Tolerance = 10^-5
+++++++++++
**************
Running experiment for A = 10.00 / 10.00
-------------


Iterations:  17%|▏| 3410/20000 [11:22<55:19,  5.00

Converged in 3411 iterations.
+++++++++++
Simulation runs Complete
++++++++++++


## Visuals

In [ ]:
plot_A_vs_capacity(As, capacities)
# plot_capcity_vs_theoretical_capacity(As, capacities, theoretical_capacitys)

A = 0.10 | Channel Capacity = 0.005
A = 0.13 | Channel Capacity = 0.008
A = 0.16 | Channel Capacity = 0.013
A = 0.21 | Channel Capacity = 0.021
A = 0.26 | Channel Capacity = 0.034
A = 0.34 | Channel Capacity = 0.053
A = 0.43 | Channel Capacity = 0.084
A = 0.55 | Channel Capacity = 0.130
A = 0.70 | Channel Capacity = 0.196
A = 0.89 | Channel Capacity = 0.285
A = 1.13 | Channel Capacity = 0.393
A = 1.44 | Channel Capacity = 0.508
A = 1.83 | Channel Capacity = 0.611
A = 2.34 | Channel Capacity = 0.736
A = 2.98 | Channel Capacity = 0.877
A = 3.79 | Channel Capacity = 1.029
A = 4.83 | Channel Capacity = 1.194
A = 6.16 | Channel Capacity = 1.372
A = 7.85 | Channel Capacity = 1.560
A = 10.00 | Channel Capacity = 1.758


In [ ]:
plot_A_vs_p_x(outputs)
plot_A_vs_p_y(outputs)

In [ ]:
# plot_p_x_records(p_x_records[idx]) # static plot
# plot_p_x_records_dynamic(outputs)
plot_p_x_records_dynamic(outputs, idx=-1)

Total Iterations: 3411
A = 10.00
Channel Capacity = 1.758
Theoretical Capacity = 1.730
Mean power = 45.795


# Saving the variables

In [ ]:
# Example data
data = {
    "As": As,
    "capacities": capacities,
    "outputs": outputs
}
timestamp = datetime.now().strftime("%Y_%m_%d_%s")
timestamp

'2024_12_01_1733035650'

In [ ]:
# Generate a timestamped file name

file_name = f"variables_{timestamp}.pkl"

# Save the file
with open(file_name, "wb") as f:
    pickle.dump(data, f)

print(f"File saved as: {file_name}")
# Output: File saved as variables_20241201_103015.pkl


File saved as: variables_2024_12_01_1733035650.pkl


In [ ]:
df = pd.DataFrame(data)
file_name = f"data_{timestamp}.csv"

df.to_csv(file_name, index=False)
print(f"CSV file saved as: {file_name}")


CSV file saved as: data_2024_12_01_1733035650.csv


# Archive

In [ ]:
def sanityCheck(outputs):
    flag = True
    print("===========")
    print("Sanity Check")
    for output in outputs:
        print(f"A = {output['A']:.2f}: sum of p_x = {np.sum(output['p_x']):.2f}, sum of p_y = {np.sum(output['p_y']):.2f}")
        print(f"sum of p(y|x) = {np.sum(output['p_y_given_x']):.2f}, sum of p_y = {np.sum(output['p_y']):.2f}")
        print("+++++++++++")

        if (np.sum(output['p_x']) > 1.01) or (np.sum(output['p_y']) > 1.01):
            flag = False
        if (np.sum(output['p_x']) < 0.99) or (np.sum(output['p_y']) < 0.99):
            flag = False
    print(f"probabilities normalized: {flag}")
    print("===========")
    return flag

In [ ]:
from scipy.stats import norm
import numpy as np

# Inputs
x = np.array([1, 2, 3])  # Example input x
y = np.array([2, 3, 4])  # Example input y
sigma = 1.0  # Standard deviation of the Gaussian noise

# Compute z-scores: (x - y) / sigma
z_scores = (x.reshape(1, -1) - y.reshape(-1, 1)) / sigma

# Compute probabilities using the standard normal PDF
p_y_given_x = norm.pdf(z_scores)

# Normalize each row to sum to 1
p_y_given_x /= p_y_given_x.sum(axis=1, keepdims=True)

# Print the result
print(p_y_given_x)

p_y_given_x.sum(axis=1)
print(z_scores)

In [ ]:
from scipy.stats import norm

# Parameters
x = 0       # Mean of the normal distribution
sigma = 1.0 # Standard deviation of the normal distribution
a = 0      # Lower bound
b = 5       # Upper bound

# Compute the probabilities using the CDF
p = norm.cdf(b, loc=x, scale=sigma) - norm.cdf(a, loc=x, scale=sigma)

# Print the result
print(f"P({a} <= y <= {b}) = {p}")